In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os
import random
import sys
import glob
sys.path.append("./")

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

#### Configuration file

In [3]:
from signet.configs.Conv1D_LSTM_CTC_Loss import Conv1D_LSTM_CTC_Loss
CFG = Conv1D_LSTM_CTC_Loss()

#### Set random seeds and training strategy

In [4]:
# Seed all random number generators
def seed_everything(seed=42):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    
def get_strategy():
    if CFG.device == "GPU"  or CFG.device=="CPU":
        ngpu = len(tf.config.experimental.list_physical_devices('GPU'))
        if ngpu>1:
            print("Using multi GPU")
            strategy = tf.distribute.MirroredStrategy()
        elif ngpu==1:
            print("Using single GPU")
            strategy = tf.distribute.get_strategy()
        else:
            print("Using CPU")
            strategy = tf.distribute.get_strategy()
            CFG.device = "CPU"

    AUTO     = tf.data.experimental.AUTOTUNE
    REPLICAS = strategy.num_replicas_in_sync
    print(f'REPLICAS: {REPLICAS}')
    
    return strategy, REPLICAS

STRATEGY, N_REPLICAS = get_strategy()

Using single GPU
REPLICAS: 1


#### Prepare dataset for training 

In [5]:
from signet.dataset.utils import get_tfrec_dataset

In [6]:
ALL_FILENAMES = glob.glob('../dataset/tdf_data/*.tfrecords')
print(len(ALL_FILENAMES))

120165


In [7]:
ROOT_DIR = "../dataset"
supp_df = pd.read_csv("../dataset/supplemental_metadata.csv")
train_df = pd.read_csv("../dataset/train.csv")

In [8]:
ds = get_tfrec_dataset(ALL_FILENAMES,CFG)
for x in ds:
    temp_train = x
    break
temp_train[1]

<unknown>
(None,)
(TensorSpec(shape=(None, 708), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))


<tf.Tensor: shape=(64, 45), dtype=int64, numpy=
array([[17, 23, 19, ..., -1, -1, -1],
       [32, 45, 35, ..., -1, -1, -1],
       [24, 18, 18, ..., -1, -1, -1],
       ...,
       [32, 45, 38, ..., -1, -1, -1],
       [32, 40, 50, ..., -1, -1, -1],
       [20, 20, 19, ..., -1, -1, -1]])>

#### Prepare model

In [3]:
import keras

ModuleNotFoundError: No module named 'keras'

In [1]:
from signet.models.feature_extractor import CNN1DFeatureExtractor

ModuleNotFoundError: No module named 'tensorflow.keras.layers.convolutional'

In [10]:
model = CNN1DFeatureExtractor(CFG)

AttributeError: module 'tensorflow.keras.layers' has no attribute 'DepthwiseConv1D'